In [1]:
import sys
from acouchbase.cluster import Cluster
from couchbase.options import ClusterOptions, SearchOptions, QueryOptions
from couchbase.auth import PasswordAuthenticator
import couchbase.collection
import couchbase.subdocument as SD
from couchbase.durability import ServerDurability, Durability
from datetime import timedelta
from couchbase.exceptions import CouchbaseException
from couchbase.search import QueryStringQuery, MatchQuery, SearchQuery, PrefixQuery, HighlightStyle, SortField, \
  SortScore, TermFacet



In [2]:
# Update this to your cluster
# Add permission to all ips and add a user database access
username = "Administrator"
password = "password"
endpoint = "localhost"
# User Input ends here.

In [3]:
# Get a reference to our cluster
# NOTE: For TLS/SSL connection use 'couchbases://<your-ip-address>' instead
cluster = Cluster('couchbases://' + endpoint + '?ssl=no_verify', ClusterOptions(PasswordAuthenticator(username,password)))
bucket = cluster.bucket('travel-sample')
collection = bucket.default_collection()

In [4]:

try:
    result = await collection.get('airline_10')
    print(result.value)

except:
    print("exception:", sys.exc_info()[0])

{'id': 10, 'type': 'airline', 'name': '40-Mile Air', 'iata': 'Q5', 'icao': 'MLA', 'callsign': 'MILE-AIR', 'country': 'Canada'}


In [5]:
query = """
  SELECT h.name, h.city, h.state
  FROM `travel-sample` h
  WHERE h.type = 'hotel'
    AND h.city = 'Malibu' LIMIT 5
"""

try:
    result = await cluster.query(query)

    for row in result:
    # each row is an instance of the query call
        name = row['name']
        city = row['city']
        print("hotel: ", name + city)

except:
    print("exception:", sys.exc_info()[0])


exception: <class 'TypeError'>


In [6]:
query = """
  SELECT h.name, h.city, h.state
  FROM `travel-sample` h
  WHERE h.type = $type
    AND h.city = $city LIMIT 5
"""

try:
    result = await cluster.query(query, type='hotel', city='Malibu')

    for row in result:
        name = row['name']
        city = row['city']
        print("Hotel: " + name + ", " + city)

except:
    print("exception:", sys.exc_info()[0])


exception: <class 'TypeError'>


In [7]:
query = """
  SELECT h.name, h.city, h.state
  FROM `travel-sample` h
  WHERE h.type = $1
    AND h.city = $2 LIMIT 5
"""

try:
    result = await cluster.query(query, 'hotel', 'Malibu')

    for row in result:
    # each row is an instance of the query call
        name = row['name']
        city = row['city']
        print("Hotel: " + name + ", " + city)

except:
    print("exception:", sys.exc_info()[0])

exception: <class 'TypeError'>


In [8]:
query = """
  SELECT h.name, h.city, h.state
  FROM `travel-sample` h
  WHERE h.type = $1
    AND h.city = $2 LIMIT 5
"""

try:
    result = await cluster.query(query, 'hotel', 'Malibu')

    for row in result:
    # each row is an instance of the query call
        name = row['name']
        city = row['city']
        print("Hotel: " + name + ", " + city)

except:
    print("exception:", sys.exc_info()[0])


exception: <class 'TypeError'>


In [9]:
try:
    result = await collection.lookup_in('airline_10', [SD.get('country')])
    country = result.content_as[str](0) # "United Kingdom"
    print("country:", country)

except:
    print("exception:", sys.exc_info()[0])


country: Canada


In [10]:
try:
    result = await collection.lookup_in('airline_10', [SD.get('country')])
    country = result.content_as[str](0) # "United Kingdom"
    print("Sub-doc before:")
    print("country:", country)

except:
    print("exception:", sys.exc_info()[0])

try:
    await collection.mutate_in("airline_10", [SD.upsert("country", "Canada")])

except:
    print("exception:", sys.exc_info()[0])

try:
    result = await collection.lookup_in('airline_10', [SD.get('country')])
    country = result.content_as[str](0) # "United Kingdom"
    print("Sub-doc after:")
    print("country:", country)

except:
    print("exception:", sys.exc_info()[0])

Sub-doc before:
country: Canada
Sub-doc after:
country: Canada


In [11]:
try:
    document = dict(
        country="Iceland", callsign="ICEAIR", iata="FI", icao="ICE",
        id=123, name="Icelandair", type="airline"
      )
    result = await collection.upsert(
        'airline_123',
        document,
        expiry=timedelta(minutes=1)
      )
    print("UPSERT SUCCESS")
    print("cas result:", result.cas)
except:
    print("exception:", sys.exc_info())

try:
    result = await collection.lookup_in('airline_123', [SD.get('name')])
    name = result.content_as[str](0) # "United Kingdom"
    print("name:", name)

except:
    print("exception:", sys.exc_info()[0])


UPSERT SUCCESS
cas result: 1680093052857745408
name: Icelandair


In [12]:
import requests
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth('Administrator', 'password')
url     = 'http://localhost:8094/api/index/ftsindex'
payload = '{"type":"fulltext-index","name":"ftsindex","sourceType":"gocbcore","sourceName":"travel-sample","planParams":{"maxPartitionsPerPIndex":1024,"indexPartitions":1},"params":{"doc_config":{"docid_prefix_delim":"","docid_regexp":"","mode":"scope.collection.type_field","type_field":"type"},"mapping":{"analysis":{},"default_analyzer":"standard","default_datetime_parser":"dateTimeOptional","default_field":"_all","default_mapping":{"dynamic":true,"enabled":false},"default_type":"_default","docvalues_dynamic":true,"index_dynamic":true,"store_dynamic":true,"type_field":"_type","types":{"inventory.hotel":{"dynamic":true,"enabled":true}}},"store":{"indexType":"scorch","segmentVersion":15}},"sourceParams":{}}'
headers = {'Content-Type': 'application/json'}
res = requests.put(url, data=payload, headers=headers, auth=auth)
res.content

b'{"error":"rest_create_index: error creating index: ftsindex, err: manager_api: cannot create index because an index with the same name already exists: ftsindex","request":{"name":"ftsindex","params":{"doc_config":{"docid_prefix_delim":"","docid_regexp":"","mode":"scope.collection.type_field","type_field":"type"},"mapping":{"analysis":{},"default_analyzer":"standard","default_datetime_parser":"dateTimeOptional","default_field":"_all","default_mapping":{"dynamic":true,"enabled":false},"default_type":"_default","docvalues_dynamic":true,"index_dynamic":true,"store_dynamic":true,"type_field":"_type","types":{"inventory.hotel":{"dynamic":true,"enabled":true}}},"store":{"indexType":"scorch","segmentVersion":15}},"planParams":{"indexPartitions":1,"maxPartitionsPerPIndex":1024},"sourceName":"travel-sample","sourceParams":{},"sourceType":"gocbcore","type":"fulltext-index"},"status":"fail"}\n'

In [13]:
#!
#!curl -XPUT -H "Content-Type: application/json" -u Administrator:password --insecure https://localhost:18094/api/index/ftsindex -d '{"type":"fulltext-index","name":"ftsindex","sourceType":"gocbcore","sourceName":"travel-sample","planParams":{"maxPartitionsPerPIndex":1024,"indexPartitions":1},"params":{"doc_config":{"docid_prefix_delim":"","docid_regexp":"","mode":"scope.collection.type_field","type_field":"type"},"mapping":{"analysis":{},"default_analyzer":"standard","default_datetime_parser":"dateTimeOptional","default_field":"_all","default_mapping":{"dynamic":true,"enabled":false},"default_type":"_default","docvalues_dynamic":true,"index_dynamic":true,"store_dynamic":true,"type_field":"_type","types":{"inventory.hotel":{"dynamic":true,"enabled":true}}},"store":{"indexType":"scorch","segmentVersion":15}},"sourceParams":{}}'

In [14]:
keys = []
try:
    result = cluster.search_query("ftsindex", QueryStringQuery("breakfast"), SearchOptions(limit=10))

    async for row in result.rows():
        keys.append(row.id)

except CouchbaseException as e:
    print("Couchbase Error:"+str(e))
except Exception as ex:
    print("Error:"+str(ex))
except:
    print("exception:", sys.exc_info()[0])


query = """
  SELECT h.*
  FROM `travel-sample` h
  USE KEYS(""" + str(keys) + """)
"""

result = cluster.query(query)
async for row in result:
    print(row['name'])

Courtyard by Marriott San Diego Downtown
Solvang Gardens Lodge
The Lord Jim Hotel
Le Bas Manoir
Country Hearth Inn
Giltar Hotel
Petite Auberge
Best Western Sonoma Valley Inn
Village Hotel
Inn at Marina del Rey


In [15]:
result = [row for row in [collection.get(key) for key in keys]]

In [ ]:
%timeit cluster.query(query).execute()

In [ ]:
async def kv_get_async(collection, key):
    return await collection.get(key)

In [19]:
import asyncio

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [20]:
loop = asyncio.get_event_loop()

In [21]:
for key in keys:
    loop.run_until_complete(collection.get(key))